In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.float_format', '{:.3f}'.format) 

In [2]:
data_2324= pd.read_csv(r"C:\Users\vaama\Downloads\NIFTY 50-07-12-2023-to-07-12-2024 (2).csv")
data_2223 = pd.read_csv(r"C:\Users\vaama\Downloads\NIFTY 50-07-12-2022-to-06-12-2023.csv")
data_2122 = pd.read_csv(r"C:\Users\vaama\Downloads\NIFTY 50-07-12-2021-to-06-12-2022.csv")
data_2021 = pd.read_csv(r"C:\Users\vaama\Downloads\NIFTY 50-07-12-2020-to-06-12-2021.csv")
data_1920 = pd.read_csv(r"C:\Users\vaama\Downloads\NIFTY 50-07-12-2019-to-06-12-2020.csv")
data = pd.concat([data_1920, data_2021,data_2122,data_2223,data_2324]) 
data['Date '] = pd.to_datetime(data['Date ']).dt.date.astype(str)

C:\Users\vaama\AppData\Local\Temp\ipykernel_28356\2645231458.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date '] = pd.to_datetime(data['Date ']).dt.date.astype(str)


In [6]:
def find_extremes(data, date, days_list):
    """
    

    
    """
    
    data = pd.DataFrame(data)
    data['Date '] = pd.to_datetime(data['Date ']).dt.date 
    date = pd.to_datetime(date).date()
    frame = pd.DataFrame()
    frame["Ref Date"] = [date]
    initial = data[data['Date '] == pd.to_datetime(date).date()] 
    initial_price = initial['Close '].max()
    frame["Intial"] = [initial_price]
    for day in days_list:
        if (day < 0):
            end_date = (pd.to_datetime(date) - pd.Timedelta(days=day)).date()
            filtered_data = data[(data['Date '] <= end_date) & (data['Date '] > pd.to_datetime(date).date())]
            highest = filtered_data['High '].max()
            percentage_increase = (highest - initial_price) / initial_price * 100
            frame[str(-day) + " day_future_max"] = [highest]
            frame[str(-day) + " day_future_percentage_increase"] = [percentage_increase]
        else:
            end_date = (pd.to_datetime(date) - pd.Timedelta(days=day)).date()
            filtered_data = data[(data['Date '] >= end_date) & (data['Date '] < pd.to_datetime(date).date())]
            lowest_past = filtered_data['Low '].min()
            highest_past = filtered_data['High '].max()
            percentage_change_max = (initial_price-highest_past) / initial_price * 100
            percentage_change_min = (initial_price - lowest_past) / initial_price * 100
            frame[str(day) + " day_past_min"] = [lowest_past]
            frame[str(day) + " day_past_percentage_change_min"] = [percentage_change_min]
            frame[str(day) + " day_past_max"] = [highest_past]
            frame[str(day) + " day_past_percentage_change_max"] = [percentage_change_max]
    
        

    return frame
    

In [4]:
window_list = [7,14,21,28,-7]

In [7]:
results = []
data['Date '] = pd.to_datetime(data['Date '])
dates = data['Date ']
data['day_of_week'] = data['Date '].dt.dayofweek
thursdays_data = data[data['day_of_week'] == 3]  # Filter Thursdays
thursdays = thursdays_data['Date '] 
# Loop through Thursdays and collect results
for weekday in thursdays:
    result = find_extremes(data, weekday,window_list)
    results.append(result)
# Concatenate all results into a single DataFrame
final_results = pd.concat(results, ignore_index=True)
print(final_results)

       Ref Date    Intial  7 day_past_min  7 day_past_percentage_change_min  \
0    2019-12-12 11971.800       11832.300                             1.165   
1    2019-12-19 12259.700       11934.000                             2.657   
2    2019-12-26 12126.550       12191.150                            -0.533   
3    2020-01-02 12282.200       12118.850                             1.330   
4    2020-01-09 12215.900       11929.600                             2.344   
..          ...       ...             ...                               ...   
246  2024-11-07 24199.350       23816.150                             1.584   
247  2024-11-14 23532.700       23509.600                             0.098   
248  2024-11-21 23349.900       23350.400                            -0.002   
249  2024-11-28 23914.150       23263.150                             2.722   
250  2024-12-05 24708.400       23873.350                             3.380   

     7 day_past_max  7 day_past_percentage_change_m

In [12]:
with pd.ExcelWriter("C:/Vaaman/Finance model predictor/processed_data.xlsx", mode='a', engine='openpyxl') as writer:
    final_results.to_excel(writer, sheet_name='Sheet5', index=False)

In [8]:
final_results.to_excel("C:/Vaaman/Finance model predictor/processed_data_20dec.xlsx", index = False)